### Transformers

Los transformers son una arquitectura de modelo en aprendizaje profundo que ha revolucionado el campo del procesamiento del lenguaje natural (NLP) y otras áreas de la inteligencia artificial. Propuestos por [Vaswani et al.](https://arxiv.org/abs/1706.03762) en 2017, los transformers utilizan mecanismos de atención para procesar secuencias de datos, eliminando la necesidad de la estructura recurrente o convolucional utilizada en modelos anteriores como RNNs y CNNs. 

La principal innovación de los transformers es su capacidad para manejar la dependencia a largo plazo de manera más eficiente y efectiva a través de la atención.



**Conceptos fundamentales**

**Codificación posicional**
  
La codificación posicional en el transformer es un mecanismo que se utiliza para introducir información sobre el orden de las palabras en una secuencia. Esto es necesario porque a diferencia de las redes neuronales recurrentes (RNNs) que procesan las secuencias de manera secuencial y tienen un conocimiento implícito del orden, los transformers procesan todas las palabras de una secuencia en paralelo, lo que significa que no tienen una forma intrínseca de capturar la posición de las palabras.

En tareas de procesamiento de lenguaje natural (NLP), el orden de las palabras en una oración es crucial para entender su significado. Por ejemplo, "el gato persigue al ratón" tiene un significado muy diferente de "el ratón persigue al gato". Sin información posicional, un transformer no podría distinguir entre estas dos oraciones, ya que vería ambas como una bolsa de palabras sin orden. La codificación posicional se suma a las embeddings de las palabras para que el modelo pueda tener información sobre la posición de cada palabra en la secuencia. La codificación sinusoidal es una de las formas en que se realiza esta codificación posicional.

La función `get_sinusoid_encoding_table(n_position, d_model)` en el siguiente código crea una tabla de codificación sinusoidal. Cada posición `pos` tiene una representación vectorial que usa funciones seno y coseno de diferentes frecuencias. Esta técnica asegura que cada posición en la secuencia tenga una representación única y que las posiciones cercanas tengan representaciones similares, capturando así las relaciones posicionales.

Una forma común de implementar la codificación posicional es mediante el uso de funciones sinusoidales. 

A continuación, se explican las ecuaciones y la lógica detrás de esta técnica:

Para una posición pos y una dimensión i en el vector de codificación de dimensiones d_model, la codificación posicional se define como:

$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$

$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$


Las funciones seno y coseno se usan para generar patrones que varían a diferentes frecuencias a lo largo de las dimensiones del vector. Las frecuencias se escalan exponencialmente con la posición y la dimensión. Se utiliza `sin` para las dimensiones pares y `cos` para las dimensiones impares. Esto asegura que cada posición tenga una representación única y que las diferencias relativas entre posiciones se codifiquen de manera consistente.

Propiedades de la codificación sinusoidal

- Las funciones sinusoidales permiten que el modelo generalice a secuencias más largas o más cortas que aquellas vistas durante el entrenamiento.
- Las representaciones de posiciones cercanas serán similares, lo cual es útil para capturar relaciones locales, mientras que las representaciones de posiciones más distantes serán suficientemente distintas.

En este ejemplo, `get_sinusoid_encoding_table` calcula la tabla de codificación sinusoidal para `n_position` posiciones y `d_model` dimensiones. La codificación posicional se aplica a la secuencia de entrada antes de pasarla al modelo Transformer, agregando estos vectores de codificación a los embeddings de las palabras.

En la arquitectura Transformer, la codificación posicional se suma a los embeddings de las palabras en la secuencia de entrada antes de ser procesados por las capas de atención. Esto se puede describir como:

$$
\text{Input Embedding} + \text{Positional Encoding}
$$

Este enfoque asegura que cada palabra en la secuencia tenga una representación que incluye tanto su identidad como su posición relativa, permitiendo que el modelo Transformer mantenga el orden de la secuencia durante el procesamiento.

**Ejercicio** Es útil visualizar las codificaciones posicionales para entender cómo varían a lo largo de las posiciones y dimensiones. Modifica el código siguiente para como se podría verse esto:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualizar codificaciones posicionales
plt.figure(figsize=(15, 5))
sns.heatmap(sinusoid_table.numpy(), cmap="viridis")
plt.xlabel('Dimensiones del Vector de Codificación')
plt.ylabel('Posiciones en la Secuencia')
plt.title('Codificaciones Posicionales Sinusoidales')
plt.show()

In [ ]:
## Tu respuesta

#### Máscara de atención

Las máscaras de atención son una parte crucial de los modelos Transformer, y se utilizan para controlar qué partes de la secuencia de entrada deben ser atendidas (prestadas atención) durante el proceso de atención. Existen principalmente dos tipos de máscaras de atención utilizadas en Transformers:

**Máscara de atención de padding**
En el procesamiento de secuencias, es común que las secuencias tengan diferentes longitudes. Para manejar esto, se rellenan las secuencias más cortas con un símbolo de padding (P). Sin embargo, estos símbolos de padding no deben influir en el proceso de atención. La máscara de atención de padding (`get_attn_pad_mask(seq_q, seq_k)`) asegura que los tokens de padding sean ignorados en el cálculo de la atención. Esta función crea una máscara que es `True` en las posiciones de padding y `False` en las demás posiciones. Al aplicar esta máscara a los puntajes de atención, se pueden asignar valores negativos muy grandes (como -1e9) a las posiciones de padding, de modo que su influencia sea minimizada.


**Máscara de atención subsecuente (causal)**

En los decodificadores de los transformers, durante el entrenamiento, es crucial que cada posición solo pueda atender a posiciones anteriores (y no a futuras) para evitar la fuga de información. La máscara de atención subsecuente (`get_attn_subsequent_mask(seq)`) se utiliza para este propósito. Esta máscara es una matriz triangular superior que asegura que la atención en una posición específica solo considere las posiciones anteriores y la posición actual, enmascarando todas las futuras posiciones.  Esto es importante en tareas de generación de texto, donde el modelo debe predecir el siguiente token sin conocer los tokens futuros.

Las máscaras de atención se aplican en la etapa de cálculo de las puntuaciones de atención (`attention scores`) para ajustar las puntuaciones de los tokens que deben ser ignorados. Los valores enmascarados se establecen en un valor muy negativo (por ejemplo, -inf) para que las puntuaciones de atención después de la aplicación de softmax sean prácticamente cero.



**Atención utilizando el producto punto escalado**

La atención es el componente clave en los transformers. El mecanismo de atención de producto punto escalado (`ScaledDotProductAttention`) calcula la importancia de cada palabra en una secuencia con respecto a otra palabra utilizando el producto punto escalado. Dado un vector de consulta `Q`, un vector de claves `K`, y un vector de valores `V`, se calcula primero el puntaje de atención como el producto punto de `Q` y `K` transpuesto, dividido por la raíz cuadrada de la dimensión de las claves `(d_k)`. Esto se hace para estabilizar los gradientes y evitar valores extremadamente grandes de la función softmax. Después, se aplica la máscara de atención (si existe) y se normalizan los puntajes utilizando una función softmax. Finalmente, se obtiene el contexto multiplicando los pesos de atención con `V`.

El mecanismo de atención de producto punto escalado calcula la relevancia entre las palabras en la secuencia. La fórmula es:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$$

Donde:
- $Q$ es la matriz de consultas.
- $K$ es la matriz de claves.
- $V$ es la matriz de valores.
- $d_k$ es la dimensión de las claves y consultas, y se usa para escalar el producto punto.

El código `ScaledDotProductAttention` implementa la atención de producto punto escalado para calcular la importancia de cada palabra en la secuencia de entrada con respecto a otra.

- La clase `ScaledDotProductAttention`: Esta clase define el mecanismo de atención de producto punto escalado, que se utiliza para calcular las puntuaciones de atención entre las consultas (`Q`), las claves (`K`) y los valores (`V`).
- El método `forward` de esta clase toma como entradas los tensores `Q`, `K` y `V`, así como una máscara de atención (`attn_mask`). A continuación, se describen los pasos que realiza el método:

In [ ]:
#scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)


- Multiplicación de matrices: Calcula el producto punto de las consultas (`Q`) y las claves (`K`). Esto da una matriz de puntuaciones que representa la afinidad entre cada par de elementos de `Q` y `K`.
- Escalado: Divide las puntuaciones por la raíz cuadrada de `d_k` (dimensión de las claves). Esto ayuda a estabilizar los gradientes y evita que las puntuaciones de atención sean demasiado grandes, lo que podría hacer que los valores softmax sean extremadamente pequeños y difíciles de manejar numéricamente.

In [ ]:
# scores.masked_fill_(attn_mask, -1e9)

Se utiliza `masked_fill_` para llenar las posiciones especificadas por la máscara de atención (`attn_mask`) con un valor muy bajo (-1e9). Este valor tan bajo asegura que después de la aplicación de la función softmax, las posiciones enmascaradas tendrán una probabilidad cercana a cero y no influirán en el resultado de la atención.

In [ ]:
# attn = nn.Softmax(dim=-1)(scores)

Se aplica la función softmax a las puntuaciones de atención a lo largo del último eje (`dim=-1`). Esto convierte las puntuaciones en probabilidades, de modo que sumen 1 en la dimensión de las claves. La función softmax resalta las puntuaciones más altas (las claves más relevantes) y atenúa las puntuaciones más bajas.

In [ ]:
#context = torch.matmul(attn, V)

Calcula el contexto como el producto punto de las probabilidades de atención (`attn`) con los valores (`V`). Esto produce una representación ponderada de los valores donde las posiciones más relevantes (según las puntuaciones de atención) contribuyen más al contexto.

In [ ]:
# return context, attn

Devuelve el contexto calculado y las puntuaciones de atención. El contexto se utilizará como entrada para las siguientes capas del transformer, y las puntuaciones de atención pueden ser útiles para interpretar qué partes de la secuencia de entrada fueron más importantes para cada predicción.

### Atención multi-cabecera

La atención multi-cabecera (`MultiHeadAttention`) es una extensión del mecanismo de atención que permite al modelo enfocarse en diferentes partes de la secuencia de entrada de manera simultánea. En lugar de realizar una única atención con grandes dimensiones, el mecanismo divide estas dimensiones en múltiples cabeceras más pequeñas y realiza el proceso de atención en paralelo. Cada cabecera de atención produce su propio conjunto de valores de contexto, que luego se concatenan y se proyectan de nuevo a la dimensión original. Esto permite al modelo capturar diferentes tipos de relaciones y patrones dentro de la secuencia. 

La función `MultiHeadAttention` define este mecanismo, utilizando capas lineales para proyectar las entradas en las subespacios de atención y luego combinar las salidas de cada cabecera.

Observación: La proyección se utiliza para mapear los datos de entrada de una dimensión a otra. En el caso de la atención multi-cabecera, la proyección se usa para transformar las consultas (`Q`), claves (`K`) y valores (`V`) de su dimensión original a una nueva dimensión que es adecuada para el cálculo de la atención.

Proceso de proyección

1. Los datos de entrada tienen una dimensión d_model.
2. Una capa lineal con parámetros (pesos W y sesgos b) transforma la entrada. La dimensión de salida de esta capa lineal es `d_k * n_heads` para las consultas y las claves, y `d_v * n_heads` para los valores.
3. La entrada se multiplica por la matriz de pesos, lo que efectúa una transformación lineal.
4. Suma del Sesgo: Se agrega el vector de sesgo para completar la transformación.
Matemáticamente, para una entrada X, la transformación lineal se expresa como: $Y=XW+b$

Donde:
- X es la entrada.
- W es la matriz de pesos.
- b es el vector de sesgo.
- Y es la salida proyectada.


In [ ]:
#def __init__(self):
#    super(MultiHeadAttention, self).__init__()
#    self.W_Q = nn.Linear(d_model, d_k * n_heads)
#    self.W_K = nn.Linear(d_model, d_k * n_heads)
#    self.W_V = nn.Linear(d_model, d_v * n_heads)
#    self.linear = nn.Linear(n_heads * d_v, d_model)
#    self.layer_norm = nn.LayerNorm(d_model)


- `self.W_Q`: Proyecta las consultas (`Q`) del espacio de dimensión `d_model` al espacio de dimensión `d_k * n_heads`.
- `self.W_K`: Proyecta las claves (`K`) del espacio de dimensión `d_model` al espacio de dimensión `d_k * n_heads`.
- `self.W_V`: Proyecta los valores (`V`) del espacio de dimensión `d_model` al espacio de dimensión `d_v * n_heads`.
- `self.linear`: Combina las salidas de todas las cabeceras de atención y las proyecta de vuelta al espacio de dimensión `d_model`.
- `self.layer_norm`: Normaliza la salida para estabilizar y acelerar el entrenamiento.

Aquí, `d_model` es la dimensión de entrada, mientras que `d_k * n_heads` y `d_v * n_heads` son las dimensiones de salida de las capas lineales para las consultas/claves y valores, respectivamente.


El método `forward` en el código define cómo se procesan las entradas a través de la capa de atención multi-cabecera. Detallemos cada paso.

In [ ]:
#q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)  # [batch_size x n_heads x len_q x d_k]
#k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)  # [batch_size x n_heads x len_k x d_k]
#v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1, 2)  # [batch_size x n_heads x len_k x d_v]

- Las consultas (`Q`), claves (`K`) y valores (`V`) se proyectan a dimensiones más altas (`d_k * n_heads`, `d_k * n_heads` y `d_v * n_heads`, respectivamente) utilizando las capas lineales `W_Q`, `W_K` y `W_V`.
- Luego, se reconfiguran las dimensiones para dividir estas proyecciones en `n_heads` diferentes cabeceras de atención.

In [ ]:
#attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)  # [batch_size x n_heads x len_q x len_k]

La máscara de atención (attn_mask) se expande y se repite para cada cabecera de atención.

In [ ]:
#context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)  # [batch_size x n_heads x len_q x d_v]


Se aplica la atención de producto punto escalado a las consultas, claves y valores divididos en múltiples cabeceras.
`ScaledDotProductAttention` calcula las puntuaciones de atención, aplica la máscara y genera el contexto.

In [ ]:
#context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v)  # [batch_size x len_q x n_heads * d_v]

Las salidas de las múltiples cabeceras se vuelven a configurar para concatenarlas en una sola representación de contexto.

In [ ]:
#output = self.linear(context)
#return self.layer_norm(output + residual), attn  # [batch_size x len_q x d_model]

* El contexto concatenado se proyecta de nuevo a la dimensión original (d_model) usando una capa lineal.
* Se aplica la normalización de capa para estabilizar el entrenamiento.
* La suma residual se utiliza para mejorar el flujo de gradientes y la estabilidad del modelo

La **autoatención**, o **self-attention**, permite que cada posición en una secuencia de entrada o salida se relacione con otras posiciones en la misma secuencia. En el transformer, la autoatención se implementa en las clases `ScaledDotProductAttention` y `MultiHeadAttention` y se utiliza tanto en el codificador como en el decodificador. Este mecanismo es fundamental para capturar dependencias a largo plazo y relaciones complejas entre palabras, mejorando así la capacidad del modelo para comprender y generar lenguaje natural.

La clase `PoswiseFeedForwardNet` en el código siguiente implementa una red de alimentación hacia adelante (Feed-Forward Network, FFN) que se aplica punto a punto (posicionalmente) en cada posición de la secuencia de entrada en un Transformer. Esta red consiste en dos capas convolucionales con una activación no lineal entre ellas, seguidas de una normalización de capa (Layer Normalization). 

In [ ]:
#def __init__(self):
#    super(PoswiseFeedForwardNet, self).__init__()
#    self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
#    self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
#    self.layer_norm = nn.LayerNorm(d_model)

- `self.conv1`: Una capa convolucional con d_model canales de entrada y `d_ff` canales de salida, y un tamaño de kernel de 1. Esto actúa como una transformación lineal que aumenta la dimensión de `d_model` a `d_ff`.
- `self.conv2`: Otra capa convolucional con `d_ff` canales de entrada y `d_model` canales de salida, y un tamaño de kernel de 1. Esto reduce la dimensión de vuelta a `d_model`.
- `self.layer_norm`: Una capa de normalización que normaliza las activaciones a lo largo de la dimensión de características d_model.

In [ ]:
#def forward(self, inputs):
#    residual = inputs  # inputs: [batch_size, len_q, d_model]
#    output = nn.ReLU()(self.conv1(inputs.transpose(1, 2)))
#    output = self.conv2(output).transpose(1, 2)
#    return self.layer_norm(output + residual)

- La entrada original se guarda en la variable residual para la conexión residual posterior.
- La entrada se transpone para que las dimensiones sean `[batch_size, d_model, len_q]`. Esto es necesario porque nn.Conv1d espera que la dimensión de características esté en el segundo lugar.
- `self.conv1` aplica una transformación lineal aumentando la dimensión de `d_model` a `d_ff`.
- La activación ReLU se aplica para introducir no linealidad en el modelo.
- Después de estos pasos, la dimensión de output es `[batch_size, d_ff, len_q]`.
- `self.conv2` aplica una segunda transformación lineal que reduce la dimensión de `d_ff` de vuelta a d_model.
-  La salida se transpone de nuevo a la forma `[batch_size, len_q, d_model]` para que coincida con la forma de la entrada original.
- La salida de las capas convolucionales se suma a la entrada original (residual), implementando una conexión residual que ayuda a mitigar el problema del desvanecimiento del gradiente y facilita el entrenamiento de redes profundas.
- La salida combinada se normaliza a lo largo de la dimensión de características d_model para estabilizar y acelerar el entrenamiento.
- La salida final del método `forward` es la representación normalizada con la suma residual aplicada.

Este mecanismo permite al transformer transformar las representaciones de manera no lineal y preservar las características importantes a través de las capas, mejorando la capacidad del modelo para aprender relaciones complejas en los datos.

La clase `EncoderLayer` define una capa del codificador en el transformer. Cada capa del codificador en un transformer consiste en dos subcomponentes principales: una capa de atención multi-cabecera y una red de alimentación hacia adelante (Feed-Forward Network, FFN). Ambas subcomponentes utilizan conexiones residuales y normalización de capa.

Por ejemplo aquí:

**Atención multi-cabecera:**

- `self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)`: Se aplica atención multi-cabecera donde las entradas `(enc_inputs)` se utilizan como consultas (Q), claves (K) y valores (V). La máscara de atención `(enc_self_attn_mask)` se aplica para ignorar los tokens de relleno.
- La salida es una tupla `(enc_outputs, attn)`, donde `enc_outputs` es el resultado de la atención y attn son las puntuaciones de atención.

**Red de alimentación hacia adelante:**

- `self.pos_ffn(enc_outputs)`: La salida de la atención se pasa a través de la red de alimentación hacia adelante.
La salida `enc_outputs` tiene la misma dimensión que la entrada debido a las conexiones residuales y la normalización de capa.

Se devuelve `enc_outputs` y las puntuaciones de atención attn.

La clase `DecoderLayer` define una capa del decodificador en el transformer. Cada capa del decodificador en un transformer consiste en tres subcomponentes principales: una capa de auto-atención multi-cabecera, una capa de atención multi-cabecera entre el decodificador y el codificador, y una red de alimentación hacia adelante (FFN). Todas estas subcomponentes utilizan conexiones residuales y normalización de capa. 

Por ejemplo aquí:

**Auto-atención multicabecera**:

- `self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)`: Se aplica auto-atención multi-cabecera donde las entradas del decodificador (dec_inputs) se utilizan como consultas (Q), claves (K) y valores (V). La máscara de auto-atención (dec_self_attn_mask) asegura que cada posición en la salida del decodificador solo puede atender a posiciones anteriores.
- La salida es una tupla `(dec_outputs, dec_self_attn)`, donde `dec_outputs` es el resultado de la auto-atención y `dec_self_attn` son las puntuaciones de auto-atención.

**Atención multi-cabecera decodificador-codificador:**

- `self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)`: Se aplica atención multi-cabecera entre el decodificador y el codificador donde las salidas del decodificador (dec_outputs) se utilizan como consultas (Q), y las salidas del codificador (enc_outputs) se utilizan como claves (K) y valores (V). La máscara de atención decodificador-codificador `(dec_enc_attn_mask)` se aplica para ignorar los tokens de relleno en el codificador.
- La salida es una tupla `(dec_outputs, dec_enc_attn)`, donde `dec_outputs` es el resultado de la atención decodificador-codificador y `dec_enc_attn` son las puntuaciones de atención.

**Red de alimentación hacia adelante:**

- `self.pos_ffn(dec_outputs)`: La salida de la atención decodificador-codificador se pasa a través de la red de alimentación hacia adelante.
- La salida dec_outputs tiene la misma dimensión que la entrada debido a las conexiones residuales y la normalización de capa.

Se devuelven `dec_outputs`, las puntuaciones de auto-atención `dec_self_attn` y las puntuaciones de atención decodificador-codificador `dec_enc_attn`.


La clase `Encoder` define el componente de codificación en un modelo Transformer. Esta clase toma una secuencia de entrada y la procesa a través de varias capas de codificación, cada una de las cuales aplica atención multi-cabecera y una red de alimentación hacia adelante posicional.

In [ ]:
#def __init__(self):
#    super(Encoder, self).__init__()
#    self.src_emb = nn.Embedding(src_vocab_size, d_model)
#    self.pos_emb = nn.Embedding.from_pretrained(get_sinusoid_encoding_table(src_len+1, d_model), freeze=True)
#    self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

- `self.src_emb:` Capa de embeddings que transforma los índices de las palabras en vectores de dimensión `d_model`.
- `self.pos_emb`: Capa de embeddings que proporciona información posicional. Se inicializa con una tabla de codificación sinusoidal preentrenada y se congela (no se entrena).
- `self.layers`: Lista de capas de codificación (EncoderLayer). Se crean `n_layers` instancias de `EncoderLayer`.


El método `forward` define cómo se procesan las entradas a través del codificador.

In [ ]:
#def forward(self, enc_inputs):  # enc_inputs: [batch_size x source_len]
#    enc_outputs = self.src_emb(enc_inputs) + self.pos_emb(torch.LongTensor([[1,2,3,4,0]]))
#    enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
#    enc_self_attns = []
#    for layer in self.layers:
#        enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
#        enc_self_attns.append(enc_self_attn)
#    return enc_outputs, enc_self_attns


- Se calculan los embeddings de las palabras y se suman a los embeddings posicionales para incorporar información de posición.
- Se crea una máscara de atención para los tokens de relleno (padding) para asegurarse de que estos no contribuyan a las puntuaciones de atención.
- Cada capa del codificador toma las salidas de la capa anterior y aplica atención multi-cabecera y una red de alimentación hacia adelante. Las puntuaciones de atención se guardan para cada capa.
- Se devuelven las salidas finales del codificador `(enc_outputs)` y las puntuaciones de atención de todas las capas `(enc_self_attns)`.

La clase `Decoder` define el componente de decodificación en un modelo transformer. Esta clase toma una secuencia de entrada del decodificador y las salidas del codificador, y las procesa a través de varias capas de decodificación.

In [ ]:
#def __init__(self):
#    super(Decoder, self).__init__()
#    self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)
#    self.pos_emb = nn.Embedding.from_pretrained(get_sinusoid_encoding_table(tgt_len+1, d_model), freeze=True)
#    self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

- `self.tgt_emb`: Capa de embeddings que transforma los índices de las palabras en vectores de dimensión `d_model`.
- `self.pos_emb`: Capa de embeddings que proporciona información posicional. Se inicializa con una tabla de codificación sinusoidal preentrenada y se congela (no se entrena).
- `self.layers`: Lista de capas de decodificación (DecoderLayer). Se crean `n_layers` instancias de `DecoderLayer`.

El método `forward` define cómo se procesan las entradas a través del decodificador.

- Se calculan los embeddings de las palabras y se suman a los embeddings posicionales para incorporar información de posición.
- Se crea una máscara de atención para los tokens de relleno (padding) en las entradas del decodificador.
- Se crea una máscara para asegurarse de que cada posición en la salida del decodificador solo pueda atender a posiciones anteriores (y no futuras).
- Se crea una máscara para ignorar los tokens de relleno en las salidas del codificador durante la atención entre el decodificador y el codificador.

In [ ]:
#dec_self_attns, dec_enc_attns = []
#for layer in self.layers:
#    dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
#    dec_self_attns.append(dec_self_attn)
#    dec_enc_attns.append(dec_enc_attn)


Cada capa del decodificador toma las salidas de la capa anterior y aplica auto-atención multi-cabecera, atención entre el decodificador y el codificador, y una red de alimentación hacia adelante. Las puntuaciones de auto-atención y atención decodificador-codificador se guardan para cada capa.


Se devuelven las salidas finales del decodificador `(dec_outputs)`, las puntuaciones de auto-atención de todas las capas `(dec_self_attns)` y las puntuaciones de atención decodificador-codificador de todas las capas `(dec_enc_attns)`.

La clase `Transformer` implementa el modelo transformer completo, que consiste en un codificador (encoder) y un decodificador (decoder).

In [ ]:
#def __init__(self):
#    super(Transformer, self).__init__()
#    self.encoder = Encoder()
#    self.decoder = Decoder()
#    self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False)

- `self.encoder`: Instancia de la clase `Encoder` que procesa las entradas de la secuencia fuente.
- `self.decoder`: Instancia de la clase `Decoder` que genera la secuencia de salida basándose en las salidas del codificador y las entradas del decodificador.
- `self.projection`: Una capa lineal que proyecta las salidas del decodificador a un espacio de dimensión igual al tamaño del vocabulario del objetivo (`tgt_vocab_size`). No se utiliza un sesgo en esta capa (`bias=False`).

El método `forward` define cómo se procesan las entradas a través del modelo transformer completo.

- `self.encoder(enc_inputs)`: Se procesan las entradas del codificador `(enc_inputs)` a través del codificador. Esto devuelve las salidas del codificador (enc_outputs) y las puntuaciones de auto-atención del codificador `(enc_self_attns)`.
- `self.decoder(dec_inputs, enc_inputs, enc_outputs)`: Se procesan las entradas del decodificador `(dec_inputs)` junto con las entradas del codificador `(enc_inputs)` y las salidas del codificador `(enc_outputs)` a través del decodificador. Esto devuelve las salidas del decodificador `(dec_outputs)`, las puntuaciones de auto-atención del decodificador `(dec_self_attns)` y las puntuaciones de atención decodificador-codificador `(dec_enc_attns)`.
- `self.projection(dec_outputs)`: Las salidas del decodificador se proyectan a un espacio de dimensión igual al tamaño del vocabulario del objetivo `(tgt_vocab_size)` utilizando una capa lineal. Esto genera los logits de decodificación `(dec_logits)`.
- `dec_logits.view(-1, dec_logits.size(-1))`: Se ajusta la forma de los logits de decodificación para que se puedan comparar con las etiquetas de la secuencia objetivo durante el cálculo de la pérdida.

Se devuelven los logits de decodificación `(dec_logits)`, las puntuaciones de auto-atención del codificador `(enc_self_attns)`, las puntuaciones de auto-atención del decodificador `(dec_self_attns)` y las puntuaciones de atención decodificador-codificador `(dec_enc_attns)`.

El código `greedy_decoder` implementa un decodificador greedy para un modelo `Transformer`. Este decodificador genera la secuencia de salida palabra por palabra durante la inferencia. El enfoque greedy significa que en cada paso se elige la palabra con la mayor probabilidad, sin considerar otras posibles secuencias.


In [ ]:
#    for i in range(0, 5):
#        dec_input[0][i] = next_symbol
#        dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs)
#        projected = model.projection(dec_outputs)
#        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
#        next_word = prob.data[i]
#        next_symbol = next_word.item()

- El bucle recorre cada posición de la secuencia de salida (de 0 a 5).
- En cada iteración, next_symbol se asigna a la posición actual en `dec_input`.
- Se pasa `dec_input` (junto con `enc_input` y `enc_outputs`) a través del decodificador del modelo para obtener `dec_outputs`, que son las representaciones decodificadas.
- Las salidas del decodificador `(dec_outputs)` se pasan por una capa de proyección para mapearlas al espacio del vocabulario del objetivo.
- `projected` se reduce a dos dimensiones, eliminando el primer eje (batch).
- `prob` es el índice de la palabra con la mayor probabilidad en cada posición.
- `next_word` se selecciona como la palabra con la mayor probabilidad en la posición actual.
- `next_symbol` se actualiza con el índice de next_word para la siguiente iteración del bucle.

La función devuelve `dec_input`, que contiene la secuencia de salida generada palabra por palabra utilizando el enfoque greedy.


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, FixedFormatter

# S: Símbolo que indica el inicio de la entrada de decodificación
# E: Símbolo que indica el inicio de la salida de decodificación
# P: Símbolo que rellenará la secuencia en blanco si el tamaño de los datos del lote actual es menor que los pasos de tiempo

def make_batch():
    # Crea el lote de entrada, salida y objetivo
    input_batch = [[src_vocab[n] for n in sentences[0].split()]]
    output_batch = [[tgt_vocab[n] for n in sentences[1].split()]]
    target_batch = [[tgt_vocab[n] for n in sentences[2].split()]]
    return torch.LongTensor(input_batch), torch.LongTensor(output_batch), torch.LongTensor(target_batch)

def get_sinusoid_encoding_table(n_position, d_model):
    # Calcula la tabla de codificación sinusoidal
    def cal_angle(position, hid_idx):
        return position / np.power(10000, 2 * (hid_idx // 2) / d_model)
    
    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(n_position)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1
    return torch.FloatTensor(sinusoid_table)

def get_attn_pad_mask(seq_q, seq_k):
    # Obtiene la máscara de atención de padding
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) es el token de PAD
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), uno es enmascarado
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

def get_attn_subsequent_mask(seq):
    # Obtiene la máscara de atención subsecuente
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequent_mask = np.triu(np.ones(attn_shape), k=1)
    subsequent_mask = torch.from_numpy(subsequent_mask).byte()
    return subsequent_mask

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9)  # Rellena elementos del tensor con valor donde la máscara es uno.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
        self.linear = nn.Linear(n_heads * d_v, d_model)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, attn_mask):
        # Q: [batch_size x len_q x d_model], K: [batch_size x len_k x d_model], V: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)  # attn_mask: [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v)  # context: [batch_size x len_q x n_heads * d_v]
        output = self.linear(context)
        return self.layer_norm(output + residual), attn  # output: [batch_size x len_q x d_model]

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, inputs):
        residual = inputs  # inputs: [batch_size, len_q, d_model]
        output = nn.ReLU()(self.conv1(inputs.transpose(1, 2)))
        output = self.conv2(output).transpose(1, 2)
        return self.layer_norm(output + residual)

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)  # enc_inputs se usa como Q, K, V
        enc_outputs = self.pos_ffn(enc_outputs)  # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs)
        return dec_outputs, dec_self_attn, dec_enc_attn

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(src_vocab_size, d_model)
        self.pos_emb = nn.Embedding.from_pretrained(get_sinusoid_encoding_table(src_len+1, d_model), freeze=True)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):  # enc_inputs: [batch_size x source_len]
        enc_outputs = self.src_emb(enc_inputs) + self.pos_emb(torch.LongTensor([[1,2,3,4,0]]))
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
        enc_self_attns = []
        for layer in self.layers:
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_emb = nn.Embedding.from_pretrained(get_sinusoid_encoding_table(tgt_len+1, d_model), freeze=True)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):  # dec_inputs: [batch_size x target_len]
        dec_outputs = self.tgt_emb(dec_inputs) + self.pos_emb(torch.LongTensor([[5,1,2,3,4]]))
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs)
        dec_self_attn_subsequent_mask = get_attn_subsequent_mask(dec_inputs)
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)

        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns

class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False)

    def forward(self, enc_inputs, dec_inputs):
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        dec_logits = self.projection(dec_outputs)  # dec_logits: [batch_size x src_vocab_size x tgt_vocab_size]
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns

def greedy_decoder(model, enc_input, start_symbol):
    """
    Para simplificar, un Decodificador Greedy es una búsqueda de Beam cuando K=1. Esto es necesario para la inferencia, ya que no 
    conocemos la secuencia de entrada objetivo. Por lo tanto, tratamos de generar la entrada objetivo palabra por palabra y luego 
    la alimentamos en el transformer. 
    Referencia inicial: http://nlp.seas.harvard.edu/2018/04/03/attention.html#greedy-decoding
    :param model: Modelo Transformer
    :param enc_input: La entrada del codificador
    :param start_symbol: El símbolo de inicio. En este ejemplo es 'S', que corresponde al índice 4
    :return: La entrada objetivo
    """
    enc_outputs, enc_self_attns = model.encoder(enc_input)
    dec_input = torch.zeros(1, 5).type_as(enc_input.data)
    next_symbol = start_symbol
    for i in range(0, 5):
        dec_input[0][i] = next_symbol
        dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs)
        projected = model.projection(dec_outputs)
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
        next_word = prob.data[i]
        next_symbol = next_word.item()
    return dec_input

def showgraph(attn):
    # Muestra el gráfico de la atención
    attn = attn[-1].squeeze(0)[0]
    attn = attn.squeeze(0).data.numpy()
    fig = plt.figure(figsize=(n_heads, n_heads))  # [n_heads, n_heads]
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attn, cmap='viridis')

    # Usando FixedLocator y FixedFormatter para evitar los warnings
    ax.set_xticks(range(len([''] + sentences[0].split())))
    ax.set_xticklabels([''] + sentences[0].split(), fontdict={'fontsize': 14}, rotation=90)
    ax.xaxis.set_major_locator(FixedLocator(ax.get_xticks()))
    ax.xaxis.set_major_formatter(FixedFormatter([''] + sentences[0].split()))

    ax.set_yticks(range(len([''] + sentences[2].split())))
    ax.set_yticklabels([''] + sentences[2].split(), fontdict={'fontsize': 14})
    ax.yaxis.set_major_locator(FixedLocator(ax.get_yticks()))
    ax.yaxis.set_major_formatter(FixedFormatter([''] + sentences[2].split()))

    plt.show()

if __name__ == '__main__':
    sentences = ['ich mochte ein bier P', 'S i want a beer', 'i want a beer E']
    # Parámetros del Transformer
    # El Padding debe ser el índice cero
    src_vocab = {'P': 0, 'ich': 1, 'mochte': 2, 'ein': 3, 'bier': 4}
    src_vocab_size = len(src_vocab)

    tgt_vocab = {'P': 0, 'i': 1, 'want': 2, 'a': 3, 'beer': 4, 'S': 5, 'E': 6}
    number_dict = {i: w for i, w in enumerate(tgt_vocab)}
    tgt_vocab_size = len(tgt_vocab)

    src_len = 5  # longitud de la fuente
    tgt_len = 5  # longitud del objetivo

    d_model = 512  # Tamaño del Embedding
    d_ff = 2048  # Dimensión de la FeedForward
    d_k = d_v = 64  # dimensión de K(=Q), V
    n_layers = 6  # número de capas del Codificador y Decodificador
    n_heads = 8  # número de cabeceras en la atención Multi-Cabecera

    model = Transformer()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    enc_inputs, dec_inputs, target_batch = make_batch()

    for epoch in range(20):
        optimizer.zero_grad()
        outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
        loss = criterion(outputs, target_batch.contiguous().view(-1))
        print('Epoca:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss))
        loss.backward()
        optimizer.step()

    # Prueba
    greedy_dec_input = greedy_decoder(model, enc_inputs, start_symbol=tgt_vocab["S"])
    predict, _, _, _ = model(enc_inputs, greedy_dec_input)
    predict = predict.data.max(1, keepdim=True)[1]
    print(sentences[0], '->', [number_dict[n.item()] for n in predict.squeeze()])

    print('Primera cabecera del último estado enc_self_attns')
    showgraph(enc_self_attns)

    print('Primera cabecera del último estado dec_self_attns')
    showgraph(dec_self_attns)

    print('Primera cabecera del último estado dec_enc_attns')
    showgraph(dec_enc_attns)


### Ejercicios


* Modifica la función `greedy_decoder` para implementar Beam Search. Ajusta el tamaño del beam y compara los resultados con la decodificación greedy.
* Implementa técnicas de regularización como dropout y weight decay en el modelo transformer. Observa cómo afectan al rendimiento y la estabilidad del entrenamiento.
* Implementa variantes de atención, como atención relativa posicional o local, y compara su impacto en la calidad de la traducción.
* Modifica el transformer para utilizar una memoria dinámica, similar al modelo Transformer-XL. Evalúa cómo esta modificación afecta a la longitud de las secuencias que el modelo puede manejar.
* Añade capas de normalización como layer normalization en diferentes partes del modelo y compara los resultados con el modelo original.
* Implementa un modelo transformer que pueda manejar múltiples modalidades de entrada, como texto e imágenes, y realiza una tarea de generación de descripciones a partir de imágenes.
* Preentrena el modelo transformer en un gran corpus de texto y luego realiza fine-tuning en un conjunto de datos específico para tareas de traducción o generación de texto.


In [ ]:
def beam_search_decoder(model, enc_input, start_symbol, beam_size=3):
    enc_outputs, enc_self_attns = model.encoder(enc_input)
    dec_inputs = torch.zeros(1, 5).type_as(enc_input.data)
    dec_inputs[0][0] = start_symbol
    end_symbol = tgt_vocab["E"]
    hypotheses = [(dec_inputs, 0.0)]  # (dec_input, log_prob)

    for i in range(1, 5):
        candidates = []
        for dec_input, log_prob in hypotheses:
            dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs)
            projected = model.projection(dec_outputs)
            probs = nn.Softmax(dim=-1)(projected[:, -1])
            topk_probs, topk_indices = probs.topk(beam_size)

            for j in range(beam_size):
                next_dec_input = dec_input.clone()
                next_dec_input[0][i] = topk_indices[0][j]
                candidates.append((next_dec_input, log_prob + torch.log(topk_probs[0][j])))

        candidates = sorted(candidates, key=lambda x: x[1], reverse=True)
        hypotheses = candidates[:beam_size]

    return hypotheses[0][0]

# Uso:
beam_dec_input = beam_search_decoder(model, enc_inputs, start_symbol=tgt_vocab["S"], beam_size=3)
predict, _, _, _ = model(enc_inputs, beam_dec_input)
predict = predict.data.max(1, keepdim=True)[1]
print(sentences[0], '->', [number_dict[n.item()] for n in predict.squeeze()])


In [ ]:
## Tus respuestas

### Otros transformers

Además del transformer original, han surgido varias variantes y extensiones que mejoran y adaptan el modelo a diferentes tareas y dominios. Algunas de las variantes más conocidas incluyen:

* BERT (Bidirectional Encoder Representations from Transformers): Un modelo preentrenado que se utiliza principalmente para tareas de clasificación y comprensión del lenguaje. BERT se entrena utilizando una técnica llamada enmascaramiento de tokens, donde algunas palabras en la secuencia de entrada se enmascaran y el modelo aprende a predecir estas palabras basándose en el contexto bidireccional.

* GPT (Generative Pre-trained Transformer): Un modelo de generación de texto que se entrena de manera autoregresiva. A diferencia de BERT, GPT se entrena para predecir la siguiente palabra en una secuencia, lo que lo hace ideal para tareas de generación de texto.

* T5 (Text-to-Text Transfer Transformer): Un modelo que convierte todas las tareas de procesamiento del lenguaje en un problema de generación de texto. T5 puede manejar una amplia variedad de tareas como traducción, resumen, y clasificación, formulando cada tarea como un problema de generación de secuencias.

* RoBERTa (Robustly optimized BERT approach): Una variante de BERT que mejora el rendimiento mediante el ajuste de los hiperparámetros de entrenamiento y el uso de un corpus más grande y diverso para el preentrenamiento.

* Transformer-XL: Una extensión que introduce conexiones recurrentes en el modelo transformer, permitiendo que maneje dependencias a largo plazo de manera más efectiva y eficiente.

* XLNet: Una variante que combina las ventajas de BERT y Transformer-XL, entrenando el modelo de manera autoregresiva y bidireccional al mismo tiempo.

Estas variantes han demostrado ser extremadamente efectivas en una amplia gama de tareas de NLP, estableciendo nuevos estándares en benchmarks y aplicaciones del mundo real.